In [1]:
import os

In [2]:
list_file = list(os.walk('./'))[0][2]
for file in list_file:
    if '替代料总表' in file:
        alternate = './' + file

print(f'\talternate: \t{alternate}')

	alternate: 	./替代料总表08-22.xlsx


In [3]:
import pandas as pd

In [4]:
df_alternate = pd.read_excel(alternate,
                             header=1)[['主料SKU', '替换料1', '替换料2', '替换料3', '替换料4', '替换料5', '替换料6']].fillna('')

df_alternate = df_alternate[df_alternate['替换料1'] != ''].reset_index(drop=True)

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6
0,USCA171876D-3,USCA171876D,USCA171876-3,USCA171876,,,
1,USGS37022T,USGS37022G,USGS37022,USGS37022J,,,
2,USGS36761TB,USGS36761,USGS36761T,,,,
3,USGS41570B,USGS41570,,,,,
4,USGS69025TB,USGS69025S,USGS69025,USGS69025T,,,
...,...,...,...,...,...,...,...
40966,USTLA85200-C,USTLA85200,,,,,
40967,USTLA89011-C,USTLA89011,,,,,
40968,USWR40588-C,USWR40588,,,,,
40969,USWR40589-C,USWR40589,,,,,


In [5]:
from tqdm import tqdm

In [6]:
count = 0
while True:
    print('# = = = = = = = ' + str(count) + ' = = = = = = = = = =')
    list_row = []
    for i in tqdm(range(len(df_alternate)), desc='运行中', ncols=77):
        list_sku = list(set(df_alternate.loc[i].tolist()))
        if '' in list_sku:
            list_sku.remove('')

        find = False
        for sku in list_sku:
            for j in range(len(list_row)):
                if sku in list_row[j]:
                    list_row[j] = list(set(list_row[j] + list_sku))
                    find = True
                    break
            if find == True:
                break

        if find == False:
            list_row.append(list_sku)  
        
    if len(list_row) == len(df_alternate):
        break
    else:
        df_alternate = pd.DataFrame(list_row).fillna('')
        df_alternate.columns = ['主料SKU'] + ['替换料'+str(i) for i in range(1, len(df_alternate.columns))]
        count += 1
        print()

# = = = = = = = 0 = = = = = = = = = =


运行中: 100%|█████████████████████████| 40971/40971 [03:00<00:00, 226.59it/s]



# = = = = = = = 1 = = = = = = = = = =


运行中: 100%|█████████████████████████| 40951/40951 [03:00<00:00, 227.16it/s]


In [7]:
import datetime

In [8]:
writer = pd.ExcelWriter('Lennon_整理好的替代料总表_' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx')
df_alternate.to_excel(writer, index=False)
writer.close()